Convolutional approach

In [1]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from tqdm import trange
import matplotlib.pyplot as plt


In [2]:
rel_path = '../trunk_sss/BIN/Debug/'

A_ref = np.loadtxt(f'{rel_path}RESULT0/Inductance_ALIJ.dat', skiprows=1, dtype=float)
PN_ref = np.loadtxt(f'{rel_path}RESULT0/ProbeVoltagePN.dat', skiprows=1, dtype=float, usecols=np.arange(1, 9))
PT_ref = np.loadtxt(f'{rel_path}RESULT0/ProbeVoltagePT.dat', skiprows=1, dtype=float, usecols=np.arange(1, 9))

In [3]:
A = []
PN = []
PT = []
for i in range(1000):
    A.append(np.loadtxt(f"{rel_path}RESULT1/A{i}.dat", skiprows=1, dtype=float))
    PN.append(np.loadtxt(f"{rel_path}RESULT1/PN{i}.dat", skiprows=1, dtype=float, usecols=np.arange(1, 9)))
    PT.append(np.loadtxt(f"{rel_path}RESULT1/PT{i}.dat", skiprows=1, dtype=float, usecols=np.arange(1, 9)))


In [4]:
Delta_PN = np.array([(x - PN_ref) for x in PN])
Delta_PT = np.array([(x - PT_ref) for x in PT])

In [9]:
Delta_PN = torch.tensor(Delta_PN).permute(0,2,1)
Delta_PT = torch.tensor(Delta_PT).permute(0,2,1)


In [10]:
# Leave only above diagonal elements
Delta_A = np.array([(x - A_ref) for x in A])
New_delta_A = []
for D_A in Delta_A:
    new = []
    for i in range(12):
        tmp = []
        for j in range(i):
            tmp.append(D_A[i][j])
        if tmp != []:
            for x in tmp:
                new.append(x)

    New_delta_A.append(new)

New_delta_A = np.array(New_delta_A)

In [11]:
Delta_IN = np.concatenate((Delta_PN, Delta_PT), axis=1)

Delta_IN.shape

(1000, 16, 161)

In [ ]:
class Net(nn.Module):
    def __init__(self, device, input_shape, output_dim):
        super(Net, self).__init__()

        self.input_shape = input_shape

        self.conv = nn.Sequential(
            nn.Conv1d(16, 4, 5, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2),
            nn.Conv1d(4, 1, 2, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2),
        ).to(device)

        conv_out = self._get_conv_out()

        self.fc = nn.Sequential(
            nn.Linear(conv_out, 2*output_dim),
            nn.ReLU(),
            nn.Linear(2*output_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, output_dim)
        ).to(device)

    def forward(self, x):
        out = self.conv(x)
        return self.fc(out.view(out.shape[0],-1))


    def _get_conv_out(self):
        o = self.conv(torch.zeros(*self.input_shape))
        return int(np.prod(o.shape[1:]))